In [2]:
import re
import requests
from requests.exceptions import ConnectionError
from http.client import RemoteDisconnected
from bs4 import BeautifulSoup
import time
import random
import json

In [ ]:
urls = [
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=carpenter',
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=driver',
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=electrician',
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=software+engineer',
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=pharmacist',
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=secretary',
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=nurse',
    'https://findajob.dwp.gov.uk/search?loc=86383&p=1&pp=50&q=childminder',

]

fileNames = [
    'carpentor',
    # 'driver',
    # 'electrician',
    # 'software_engineer',
    # 'pharmacist',
    # 'secretary',
    # 'nurse',
    # 'childminder'
    ]

In [16]:
jobNumberFinder = 'govuk-heading-l'
jobTitleFinder = "govuk-heading-s govuk-!-margin-top-4 govuk-!-margin-bottom-2"
jobDetailDescriptionFinder = 'govuk-body govuk-!-margin-bottom-6'

In [ ]:
# 後で削除
import socks,socket
from urllib.request import urlopen
from fake_useragent import UserAgent

userAgent = UserAgent().random
print(userAgent)

In [29]:
titles = []
titleAreas = soup.find_all("h3", class_=jobTitleFinder)
for titleArea in titleAreas:
    titles.append(titleArea.findChildren("a" , recursive=False))


In [31]:
Response = requests.get(titles[0][0]['href'])

In [38]:

jobDetailSoup = BeautifulSoup(Response.content, "html.parser")
print(jobDetailSoup)


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Carpenters</title>
<link href="//static.findajob.dwp.gov.uk/dist/jobseeker/Jobseeker.bundle-3.7.0-vsn.css" rel="stylesheet"/>
<link href="//static.findajob.dwp.gov.uk/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
</head>
<body class="govuk-template__body page_ad_details">
<script type="application/ld+json">
{
   "@context" : "http://schema.org",
   "@type" : "JobPosting",
   "baseSalary" : {
      "@type" : "MonetaryAmount",
      "currency" : "GBP",
      "minValue" : "34560"
   },
   "datePosted" : "2022-01-18",
   "description" : "Carpenters required for on-goingcontract work nearWest Drayton, \u00a318.00p/h.<br /><br /> 2nd carpentryfix work .Carpenters on this job will need their own tools. Plenty of free onsite parking,<br /><br /> Immediate starts available Carpenter / Joiners, please click to apply!",
   "employmentType" :

In [ ]:
data = []
index = 0
failedTopUrls = []
failedDetailUrls = []
dataIndex = 1
pageNumber = 1
for url in urls:
    try:
        # 後で削除 
        socks.setdefaultproxy(proxy_type=socks.PROXY_TYPE_SOCKS5, addr='127.0.0.1', port=9050)
        socket.socket = socks.socksocket
        headers = {'User-Agent': UserAgent().random}
        response = requests.get(url, headers=headers)
        # 後で削除
        # get number of jobs for the occupation 
        # response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        jobNumberText = soup.find("h1", class_=jobNumberFinder).text
        jobNumber = re.sub(r'\D', '', jobNumberText) if jobNumberText is not None else 0
        print("Success - url:" + url + "jobNumber:" + jobNumber)
    except ConnectionError:
        print('Fail - url:' + url)
        failedTopUrls.append(url)
        time.sleep(0.5)
        continue
    while(True):
        if(pageNumber == 1):
            titles = []
            titleAreas = soup.find_all("h3", class_=jobTitleFinder)
            for titleArea in titleAreas:
                titles.append(titleArea.findChildren("a" , recursive=False))
        else:
            try:
                newUrl = url.replace('&p\=[0-9]', '&p=' + str(pageNumber))
                # 後で削除
                headers = {'User-Agent': UserAgent().random}
                response = requests.get(newUrl, headers=headers)
                # 後で削除
                # get list of jobs for each urls 
                # response = requests.get(url)
                soup = BeautifulSoup(response.content, "html.parser")
                time.sleep(0.5)
                response.close()
                titles = []
                titleAreas = soup.find_all("h3", class_=jobTitleFinder)
                for titleArea in titleAreas:
                    titles.append(titleArea.findChildren("a" , recursive=False))
            except ConnectionError:
                print('Fail - url:' + newUrl)
                failedTopUrls.append(url)
                pageNumber = pageNumber + 1
                if(int(jobNumber) >= pageNumber * 50):
                    break
                time.sleep(0.5)
                continue
        for title in titles:
            try:
                # 後で削除
                headers = {'User-Agent': UserAgent().random}
                jobDetailResponse = requests.get(title[0]['href'], headers=headers)
                # 後で削除
                # get list of jobs for each urls 
                # jobDetailResponse = requests.get(title[0]['href'])
                jobDetailSoup = BeautifulSoup(jobDetailResponse.content, "html.parser")
                jobDetailResponse.close()
                time.sleep(0.5)
                jobdescription = None if jobDetailSoup.find("div", class_=jobDetailDescriptionFinder) is None else jobDetailSoup.find("div", class_=jobDetailDescriptionFinder).text
                data.append({str(dataIndex) : {'title': title.text, 'description':jobdescription}})
                dataIndex = dataIndex + 1
            except ConnectionError:
                print('Fail - detail url:' + url)
                failedDetailUrls.append(url)
                time.sleep(0.5)
                continue
            time.sleep(random.randint(2,7))
        pageNumber = pageNumber + 1
        if(int(jobNumber) >= pageNumber * 50):
            break
        time.sleep(random.randint(2,7))
    
    # export data into json file
    with open('./data/en/' + fileNames[index] + '.json', mode='w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)
        print("Success - dump " + fileNames[index] + '.json')
    index = index + 1

# export Top Url data into json file
with open('./data/en/failedTopUrls.json', mode='w', encoding='utf-8') as file:
    json.dump(failedTopUrls, file, ensure_ascii=False, indent=2)
    print("Success - dump " + 'failedTopUrls.json')
    
# export detailed Url data into json file
with open('./data/en/failedDetailUrls.json', mode='w', encoding='utf-8') as file:
    json.dump(failedDetailUrls, file, ensure_ascii=False, indent=2)
    print("Success - dump " + 'failedDetailUrls.json')
    